# More Monad

## Writer

In [1]:
isBigGang :: Int -> Bool
isBigGang x = x > 9

In [2]:
-- 로그를 같이 찍고 싶다면?
isBigGang :: Int -> (Bool, String)
isBigGang x = (x>9, "Compared gang size to 9")

In [3]:
isBigGang 3
isBigGang 30

(False,"Compared gang size to 9")

(True,"Compared gang size to 9")

In [4]:
applyLog :: (a, String) -> (a -> (b, String)) -> (b, String)
applyLog (x, log) f = let (y, newLog) = f x in (y, log ++ newLog)

In [5]:
(3, "smallish gang.") `applyLog` isBigGang
(30, "a freaking platoon") `applyLog` isBigGang

(False,"smallish gang.Compared gang size to 9")

(True,"a freaking platoonCompared gang size to 9")

In [7]:
("tobin", "Got outlaw name.") `applyLog` (\x -> (length x, "applied lenght."))
("Bathcat","Got outlaw name.") `applyLog` (\x -> (length x, "Applied length")) 

(5,"Got outlaw name.applied lenght.")

(7,"Got outlaw name.Applied length")

In [16]:
import qualified Data.ByteString as B

[1,2,3] `mappend` [4,5,6]
-- list ++
B.pack [99, 104,105] `mappend` B.pack [104,117,97,104,117,97]
-- string ++

-- 모노이드 값이라는 것을 안 순간, 모노이드로써 처리하게 된다

[1,2,3,4,5,6]

"chihuahua"

In [20]:
applyLog :: (Monoid m) => (a,m) -> (a -> (b,m)) -> (b,m)  
applyLog (x,log) f = let (y,newLog) = f x in (y,log `mappend` newLog)  

In [22]:
import Data.Monoid

type Food = String
type Price = Sum Int
:info Sum

addDrink ::  Food -> (Food, Price)
addDrink "beans" = ("milk", Sum 25)
addDrink "jerky" = ("whiskey", Sum 99)
addDrink _  = ("beer", Sum 30)

type Sum :: * -> *
newtype Sum a = Sum {getSum :: a}
  	-- Defined in ‘base-4.14.1.0:Data.Semigroup.Internal’
instance Num a => Num (Sum a) -- Defined in ‘base-4.14.1.0:Data.Semigroup.Internal’
instance Functor Sum -- Defined in ‘base-4.14.1.0:Data.Semigroup.Internal’
instance Show a => Show (Sum a) -- Defined in ‘base-4.14.1.0:Data.Semigroup.Internal’
instance Eq a => Eq (Sum a) -- Defined in ‘base-4.14.1.0:Data.Semigroup.Internal’
instance Num a => Monoid (Sum a) -- Defined in ‘base-4.14.1.0:Data.Semigroup.Internal’
instance Ord a => Ord (Sum a) -- Defined in ‘base-4.14.1.0:Data.Semigroup.Internal’
instance Num a => Semigroup (Sum a) -- Defined in ‘base-4.14.1.0:Data.Semigroup.Internal’
instance Read a => Read (Sum a) -- Defined in ‘base-4.14.1.0:Data.Semigroup.Internal’
instance Applicative Sum -- Defined in ‘base-4.14.1.0:Data.Semigroup.Internal’
instance Foldable Sum -- Defined in ‘Data.Foldable’
instance Monad Sum -- Defined in ‘base-4.14.1.0:Data.Semigroup.Internal’
instance Traversable Sum -- Defined in ‘Data.Traversable’
instance Bounded a => Bounded (Sum a) -- Defined in ‘base-4.14.1.0:Data.Semigroup.Internal’

In [23]:
Sum 3 `mappend` Sum 9

Sum {getSum = 12}

In [24]:
("beans", Sum 10) `applyLog` addDrink
("jerky", Sum 25) `applyLog` addDrink
("meat", Sum 5) `applyLog` addDrink

("milk",Sum {getSum = 35})

("whiskey",Sum {getSum = 124})

("beer",Sum {getSum = 35})

In [ ]:
newtype Writer w a = Writer { runWriter :: (a, w) }  
-- a는 값의 타입
-- w는 추가된 모노이드 값의 타입
-- Writer 생성자는 export 되지 않음

instance (Monoid w) => Monad (Writer w) where  
    return x = Writer (x, mempty)  
    (Writer (x,v)) >>= f = let (Writer (y, v')) = f x in Writer (y, v `mappend` v')  

In [37]:
import qualified Control.Monad.Writer as Writer

Writer.runWriter (return 3 :: Writer.Writer String Int)
Writer.runWriter (return 3 :: Writer.Writer (Sum Int) Int)
Writer.runWriter (return 3 :: Writer.Writer (Product Int) Int)

(3,"")

(3,Sum {getSum = 0})

(3,Product {getProduct = 1})

In [42]:
-- 모나드로 동작한다는 사실을 알았으니, 모나드로 작성한다.

import Control.Monad.Writer

logNumber :: Int -> Writer [String] Int
logNumber x = writer (x, ["Got number: " ++ show x])

multWithLog :: Writer [String] Int
multWithLog = do
    a <- logNumber 3
    b <- logNumber 5
    tell ["Gonna multiply these two"]
    -- 컨텍스트 값만 추가한다.
    return (a*b)
    
multWithLog
runWriter multWithLog

WriterT (Identity (15,["Got number: 3","Got number: 5","Gonna multiply these two"]))

(15,["Got number: 3","Got number: 5","Gonna multiply these two"])

In [43]:
gcd' :: Int -> Int -> Int
gcd' a b
    | b == 0 = a
    | otherwise = gcd' b (a `mod` b)

In [44]:
gcd' 8 3

1

In [47]:
-- 로그 추가
import Control.Monad.Writer  
  
gcd' :: Int -> Int -> Writer [String] Int  
gcd' a b  
    | b == 0 = do  
        tell ["Finished with " ++ show a]  
        return a  
    | otherwise = do  
        tell [show a ++ " mod " ++ show b ++ " = " ++ show (a `mod` b)]  
        gcd' b (a `mod` b)  

In [48]:
fst $ runWriter (gcd' 8 3)

1

In [49]:
mapM_ putStrLn $ snd $ runWriter (gcd' 8 3)

8 mod 3 = 2
3 mod 2 = 1
2 mod 1 = 0
Finished with 1

## 비효율적인 리스트 구조

In [50]:
import Control.Monad.Writer  
  
gcdReverse :: Int -> Int -> Writer [String] Int  
gcdReverse a b  
    | b == 0 = do  
        tell ["Finished with " ++ show a]  
        return a  
    | otherwise = do  
        result <- gcdReverse b (a `mod` b)  
        -- 후 콘텍스트를 먼저 받음
        tell [show a ++ " mod " ++ show b ++ " = " ++ show (a `mod` b)]  
        return result  
        -- ++의 사용을 오른쪽이 아닌 왼쪽에 연결하면서 끝나기 때문에 비효율적

In [51]:
mapM_ putStrLn $ snd $ runWriter (gcdReverse 8 3)

Finished with 1
2 mod 1 = 0
3 mod 2 = 1
8 mod 3 = 2

In [ ]:
-- Difference list
-- 사실 함수이다

-- 일반 빈 리스트
[]
-- 빈 디퍼런스 리스트 (함수)
\xs -> [] ++ xs

-- 두개의 디퍼런스 리스트를 붙이는 것은 다음과 같다
f `append` g = \xs -> f (g xs)

In [54]:
-- 효율적인 ++ 순서를 보장하기 위해서 새로운 타입 선언
newtype DiffList a = DiffList {getDiffList :: [a] -> [a]}

toDiffList :: [a] -> DiffList a  
toDiffList xs = DiffList (xs++)  
  
fromDiffList :: DiffList a -> [a]  
fromDiffList (DiffList f) = f []  

instance Monoid (DiffList a) where  
    mempty = DiffList (\xs -> [] ++ xs)  
    (DiffList f) `mappend` (DiffList g) = DiffList (\xs -> f (g xs))  

Line 11: Avoid lambda
Found:
(\ xs -> [] ++ xs)
Why not:
([] ++)Line 11: Evaluate
Found:
[] ++ xs
Why not:
xsLine 12: Avoid lambda
Found:
\ xs -> f (g xs)
Why not:
f . g

: 

## Reader

In [57]:
fmap (*5) (+3) 8

55

In [63]:
f' = (+) <$> (*2) <*> (+10)
f'' = \x -> (x*2) + (x + 10)

f' 3
f'' 3


-- foo = (+) <$> (*2) 
-- -- (\x y -> (x*2) + y)
-- :t foo 
-- foo 3 4

-- bar = foo <*> (+10)
-- -- :t foo :: Num a => a -> a -> a :: (-> a) a -> a
-- -- :t (+10) :: Num a => a -> a :: (-> a) a 
-- -- f <*> g = \x -> f x (g x)  
-- \x -> foo x (x + 10)
-- bar = \x -> (x*2) + (x + 10)

Line 2: Redundant lambda
Found:
f'' = \ x -> (x * 2) + (x + 10)
Why not:
f'' x = (x * 2) + (x + 10)

19

19

In [65]:
addThree x y z = x + y + z
g' = addThree <$> (*2) <*> (+10) <*> (/10)
g' 20
((20*2) + (20+10) + (20/10))

Line 4: Redundant bracket
Found:
((20 * 2) + (20 + 10) + (20 / 10))
Why not:
(20 * 2) + (20 + 10) + (20 / 10)

72.0

72.0

In [ ]:
instance Monad ((->) r) where  
    return x = \_ -> x  
    -- x :: a (함수의 반환값)
    -- return :: a -> m a
    h >>= f = \w -> f (h w) w  
    -- h :: m a :: (r -> a)
    -- 첫번째 인자 : 일반 함수
    -- f :: (a -> m b) :: (a -> r -> b)
    -- 두번째 인자 : 모나드 내부 값을 받아 모나드 값을 반환
    -- w :: r
    -- h >>= f :: (r -> a) -> (a -> r -> b) -> (r -> b)
    --         :: (m a) -> (a -> m b) -> m b

In [70]:
foo = (+ 3) >>= (\x y-> (x - y) * y)
-- \w -> 3 * w
:t foo
foo 1

foo :: forall b. Num b => b -> b

3

In [108]:
import Control.Monad.Instances

addStuff :: Int -> Int
addStuff = do
    a <- (*2)
    b <- (+10)
    -- 바인딩 값이 함수가 무엇을 반환할것인지에 대해서 이미 알고 있는 것처럼 행동하게 됨
    -- 함수들을 하나의 함수로 합치고,
    -- 그 함수의 매개변수를 그것을 구성하는 모든 함수들에게 전달하는 것으로 이루어진다.
    return (a + b)
    -- 리턴이 함수를 반환하는 함수이므로, 인자가 두 개인 것처럼 인식됨
    
addStuff' = (*2) >>= (\a -> (+ 10) >>= (\b -> return (a + b)))
-- h >>= f = \w -> f (h w) w  
    
addStuff 4
addStuff' 4

22

22

In [107]:
(do a <- (+2); return a) $ 5
(+2) >>= (\a -> return a) $ 5
(\w -> (return (w + 2)) w ) 5
(\w -> (\_ -> w + 2) w) $ 5
(\w -> w + 2) $ 5

Line 1: Redundant return
Found:
do a <- (+ 2)
   return a
Why not:
do (+ 2)Line 1: Redundant $
Found:
(do a <- (+ 2)
    return a)
  $ 5
Why not:
(do a <- (+ 2)
    return a)
  5Line 2: Avoid lambda
Found:
(\ a -> return a)
Why not:
returnLine 3: Redundant bracket
Found:
(return (w + 2)) w
Why not:
return (w + 2) wLine 4: Redundant $
Found:
(\ w -> (\ _ -> w + 2) w) $ 5
Why not:
(\ w -> (\ _ -> w + 2) w) 5Line 5: Avoid lambda using `infix`
Found:
(\ w -> w + 2)
Why not:
(+ 2)Line 5: Redundant $
Found:
(\ w -> w + 2) $ 5
Why not:
(\ w -> w + 2) 5

7

7

7

7

7

In [109]:
addStuff :: Int -> Int
addStuff x = let
    a = (*2) x
    b = (+10) x
    in a + b
    
addStuff 4
-- 더 명확하게 작성

22

## 세련되게 상태를 유지하는 계산

State Monad

s -> (a,s)

s :: 상태의 타입
a :: 스테이트풀 계산의 결과

## stack

- Push
- Pop

In [2]:
type Stack = [Int]

pop :: Stack -> (Int, Stack)
pop (x:xs) = (x,xs)

push :: Int -> Stack -> ((), Stack)
push a xs = ((), a:xs)

In [113]:
stackManip :: Stack -> (Int, Stack)
stackManip stack = let
    ((), newStack1) = push 3 stack
    (a, newStack2) = pop newStack1
    in pop newStack2

In [115]:
stackManip [5,8,2,1]
-- Push [3,5,8,2,1]
-- Pop [5,8,2,1]
-- Pop [8,2,1]

(5,[8,2,1])

In [47]:
stackManip = do
    push 3
    a <- pop
    pop

:t stackManip

-- Monad (Int, a) 
-- pop :: Stack -> (Int, Stack)
-- push :: Int -> Stack -> ((), Stack)

stackManip' = push 3 >> pop >>= (\a -> pop)

:t stackManip'

stackManip :: Stack -> (Int, Stack)

stackManip' :: Stack -> (Int, Stack)

In [48]:
stackManip [5,8,2,1]
stackManip' [5,8,2,1]

(5,[8,2,1])

(5,[8,2,1])

## State Monad

In [126]:
-- Control.Monad.State

newtype State s a = State {runState :: s -> (a, s)}

instance Monad (State s) where  
    return x = State $ \s -> (x,s)  
    -- 인자 값을 항상 state로 반환하는 "계산"을 만든다.
    -- x 가 계산의 결과
    (State h) >>= f = let (a, newState) = h s  
                                        (State g) = f a  
                                    in  g newState  
    -- 첫번째 인자는 State 값 (연산)이다
    -- h :: s -> (a,s) (deconstructing)
    -- f :: a -> State (s -> (a,s)) 
    -- State s를 컨텍스트로 보면 컨텍스트를 벗겨낸 값은 a이다.
    -- 전체 반환값은 모나드이다. (당연하다)
    
    -- h 는 기존의 State 컨텍스트 계산
    -- f 는 a -> State (s -> (a,s))
    
    -- (a, newState) 는 기존 계산에 state를 적용해서 나온 튜플
    -- g는 f에 a를 적용해서 나온 계산 :: (s -> (a,s))
    -- g newState는 :: (a,s)
    
    let (a, newState) = h s  
        -- 기존 계산에 최종 인자 state를 적용하여 결과과 새 state를 받음
        (State g) = f a  
        -- f에 위에서 받은 결과를 적용하여 g 계산을 받는다.
    in  g newState  
    -- 받은 계산 g에 newState를 적용하여 (h s 와 느낌 비슷)
    -- (a, s) 받음
    -- == 새로운 계산과 새로운 상태를 적용
    
    State $ \s -> -- g newState
    -- 받은 (a,s)를 통해 State Monad 생성

In [51]:
import qualified Control.Monad.State as S

type Stack = [Int]

-- State 모나드는 계산이다.

pop :: S.State Stack Int
-- State (Stack -> (Int, Stack))
pop = S.state $ \(x:xs) -> (x,xs)

push :: Int -> S.State Stack ()
-- State (Stack -> ((), Stack))
push a = S.state $ \xs -> ((), a:xs)

In [53]:
import qualified Control.Monad.State as S

stackManip :: S.State Stack Int
stackManip = do
    push 3
    a <- pop
    -- Monad (State s) 이므로 a는 계산 결과값이다.
    pop

In [55]:
import qualified Control.Monad.State as S

S.runState stackManip $ [5,8,2,1]

Line 3: Redundant $
Found:
S.runState stackManip $ [5, 8, 2, 1]
Why not:
S.runState stackManip [5, 8, 2, 1]

(5,[8,2,1])

In [56]:
stackManip' :: S.State Stack Int
stackManip' = push 3 >> pop >>= (\a -> pop)

push 3 >> pop
-- push 3 >>= \_ -> pop
-- (State \xs -> ((), 3:xs)) >>= \_ -> pop

-- h = \xs -> ((), 3:xs)
-- f = \_ -> pop

-- let (a, newState) = h s
-- let (a, newState) = ((), 3:s)
-- a = ()
-- newState = 3:s

-- (State g) = f a
-- (State g) = (\_ -> pop) ()
-- (State g) = pop
-- g = \(x:xs) -> (x,xs)

-- in g newState
-- in (\(x:xs) -> (x,xs)) (3:s)
-- in (3, s)

-- == push 3 >> pop = State \s -> (3,s)

-- (State h) >>= f = let (a, newState) = h s  
--                       (State g) = f a  
--                   in  g newState  

Line 2: Use const
Found:
\ a -> pop
Why not:
const pop

In [59]:
S.runState stackManip $ [5,8,2,1]
S.runState stackManip' $ [5,8,2,1]

Line 1: Redundant $
Found:
S.runState stackManip $ [5, 8, 2, 1]
Why not:
S.runState stackManip [5, 8, 2, 1]Line 2: Redundant $
Found:
S.runState stackManip' $ [5, 8, 2, 1]
Why not:
S.runState stackManip' [5, 8, 2, 1]

(5,[8,2,1])

(5,[8,2,1])

In [60]:
import qualified Control.Monad.State as S

stackManip :: S.State Stack Int
stackManip = do
    push 3
    pop
    -- 다음 연산에서 pop의 결과 값읋 사용하지 않으므로 굳이 꺼낼 필요가 없다.
    pop

In [61]:
import qualified Control.Monad.State as S

stackStuff :: S.State Stack ()
stackStuff = do
    a <- pop
    if a == 5
        then push 5
        else do
            push 3
            push 8

In [62]:
import qualified Control.Monad.State as S

S.runState stackStuff [9,0,2,1,0]

((),[8,3,0,2,1,0])

In [64]:
v

moreStack :: S.State Stack ()
moreStack = do
    a <- stackManip
    if a == 100
        then stackStuff
        else return ()

Line 6: Use when
Found:
if a == 100 then stackStuff else return ()
Why not:
Control.Monad.when (a == 100) stackStuff

### State 얻기와 설정하기

In [ ]:
-- 구현체
get = State $ \s -> (s,s) 
put newState = State $ \s -> ((),newState)  

In [67]:
import qualified Control.Monad.State as S

stackyStack :: S.State Stack ()  
stackyStack = do  
    stackNow <- S.get  
    -- 결과 값으로 상태를 받음
    if stackNow == [1,2,3]  
        then S.put [8,3,1]  
        else S.put [9,2,1]  
        -- 상태를 그냥 바꿔 버림

In [69]:
S.runState stackyStack [1,2,3]
S.runState stackyStack [1,2,3,4]

((),[8,3,1])

((),[9,2,1])

In [71]:
:t (>>=)

-- (>>=) at (State s) :: (State s a) -> (a -> State s b) -> State s b

(>>=) :: forall (m :: * -> *) a b. Monad m => m a -> (a -> m b) -> m b

## 난수와 State Monad

In [72]:
import qualified System.Random as R

:t R.random
-- (Random a, RandomGen g) => g -> (a, g)

R.random :: forall a g. (Random a, RandomGen g) => g -> (a, g)

In [75]:
import System.Random
import Control.Monad.State

randomSt :: (RandomGen g, Random a) => State g a
randomSt = state random
-- g(enerator)가 state(context)가 되며, a는 결과(result)가 된다.

threeCoins :: State StdGen (Bool, Bool, Bool)
-- State (runState :: StdGen -> ((Bool, Bool, Bool), StdGen))
threeCoins = do
    a <- randomSt
    b <- randomSt
    c <- randomSt
    return (a,b,c)

runState threeCoins (mkStdGen 33)

((True,False,True),680029187 2103410263)

## 에러

In [2]:
:t Right 4
:t Left "out of cheese error"

Right 4 :: forall b a. Num b => Either a b

Left "out of cheese error" :: forall b. Either [Char] b

In [ ]:
instance (Error e) => Monad (Either e) where  
    return x = Right x   
    -- 최소 콘텍스트 (right 는 성공이다)
    Right x >>= f = f x  
    Left err >>= f = Left err  
    fail msg = Left (strMsg msg)  

In [3]:
import Control.Monad.Error (strMsg)

:t strMsg
strMsg "Boom!" :: String

strMsg :: forall a. Error a => String -> a


<interactive>:1:1: warning: [-Wdeprecations] In the use of ‘strMsg’ (imported from Control.Monad.Error, but defined in Control.Monad.Trans.Error): Deprecated: "Use Control.Monad.Trans.Except instead"
"Boom!"

In [5]:
Left "boom" >>= \x -> return (x + 1)
Left "boom" >>= \x -> Left "no way!"
Right 100 >>= \x -> Left "no way!"

Left "boom"

Left "boom"

Left "no way!"

In [6]:
Right 3 >>= \x -> return (x + 100)

Right 103

In [7]:
Right 3 >>= \x -> return (x + 100) :: Either String Int

Right 103

## 유용한 모나드 함수

In [ ]:
-- monad is applicative functor
-- applicative is functor

### liftM

In [10]:
import Control.Monad

:t liftM
-- liftM :: forall (m :: * -> *) a1 r. Monad m => (a1 -> r) -> m a1 -> m r
-- liftM :: (Monad m) => (a -> b) -> m a -> m b
:t fmap
-- fmap :: (Functor f) => (a -> b) -> f a -> f b

-- 모나드를 위한 fmap이라고 이해하자

liftM :: forall (m :: * -> *) a1 r. Monad m => (a1 -> r) -> m a1 -> m r

fmap :: forall (f :: * -> *) a b. Functor f => (a -> b) -> f a -> f b

In [40]:
import Control.Monad
import qualified Control.Monad.Writer as W
import qualified Control.Monad.State as S

liftM (*3) (Just 8)
-- (Just 8) >>= (*3) -- >>= 의 함수인자는 반환을 Monad 타입으로 해야한다. (오류남)
-- liftM function arg returning not monad
fmap (*3) (Just 8)
W.runWriter $ liftM not $ W.writer (True, "chickpeas")
W.runWriter $ fmap not $ W.writer (True, "chickpeas")
-- Writer is functor and monad
-- S.runState (liftM (+100) pop) [1,2,3,4]
-- S.runState (fmap (+100) pop) [1,2,3,4]

Line 5: Use fmap
Found:
liftM
Why not:
fmapLine 9: Use fmap
Found:
liftM
Why not:
fmapLine 10: Use <$>
Found:
fmap not $ W.writer (True, "chickpeas")
Why not:
(not <$> W.writer (True, "chickpeas"))

Just 24

Just 24

(False,"chickpeas")

(False,"chickpeas")

In [ ]:
-- liftM의 구현 방법
liftM :: (Monad m) => (a -> b) -> m a -> m b  
liftM f m = m >>= (\x -> return (f x))  

liftM :: (Monad m) => (a -> b) -> m a -> m b  
liftM f m = do  
    x <- m  
    return (f x)  

### ap 함수

In [ ]:
-- ap 함수의 정의
ap :: (Monad m) => m (a -> b) -> m a -> m b  
ap mf m = do  
    f <- mf  
    x <- m  
    return (f x)  

In [45]:
Just (+3) <*> Just 4
Just (+3) `ap` Just 4
[(+1), (+2), (+3)] <*> [10,11]
[(+1), (+2), (+3)] `ap` [10,11]

Just 7

Just 7

[11,12,12,13,13,14]

[11,12,12,13,13,14]

### liftA2 함수

편의함수이다

어플리커티브 클래스 제약이 있다.
A 뒤의 숫자에 따라 인자가 달라지며
A가 아니라 M일 때는 Monad를 받는다.

In [ ]:
liftA2 :: (Applicative f) => (a -> b -> c) -> f a -> f b -> f c  
liftA2 f x y = f <$> x <*> y  

### join 함수

In [ ]:
-- join의 타입
join :: (Monad m) => m (m a) -> m a  
-- 모나드 모나드 a -> 모나드 a
-- 즉슨, 모나드 값을 가진 모나드 값을 받아서 하나의 모나드 값을 반환한다.

In [46]:
join (Just (Just 9))
join (Just Nothing)
join (Nothing)
-- 실패 컨텍스트를 벗기려고 해서 결과는 바로 Nothing이 되었다.

Line 3: Redundant bracket
Found:
(Nothing)
Why not:
Nothing

Just 9

Nothing

Nothing

In [47]:
join [[1,2,3], [4,5,6]]

[1,2,3,4,5,6]

In [49]:
import Control.Monad.Writer as W

W.runWriter $ join (W.writer (W.writer (1, "aaa") , "bbb") )

-- 로그를 바깥쪽에서부터 안쪽으로 쓴다.

(1,"bbbaaa")

In [53]:
join (Right (Right 9)) :: Either String Int
join (Right (Left "error")) :: Either String Int
join (Left "error2") :: Either String Int
-- Maybe 실패 컨텍스트의 형태가 join과 비슷하다.

Right 9

Left "error"

Left "error2"

In [ ]:
runState (join (State $ \s -> (push 10,1:2:s))) [0,0,0]  
-- ((),[10,1,2,0,0,0])  

In [ ]:
-- join에 대한 구현체
join :: (Monad m) => m (m a) -> m a  
join mm = do  
    m <- mm  
    m  

In [56]:
joinedMaybes :: Maybe Int
joinedMaybes = do
    m <- Just (Just 8)
    m
joinedMaybes

Line 2: Use join
Found:
do m <- Just (Just 8)
   m
Why not:
do join (Just (Just 8))

Just 8

In [ ]:
-- 모나드 값을 >>= 에 연결하는 것은
-- 모나드 값을 일단 join 을 사용하는 것 과 동일하다
-- m >>= f 는 항상 join (fmap f m) 과 동일하다
-- fmap은 모나드 값을 반환하므로 위 식은 모나드 모나드 값을 반환한다.

### filterM

여기서 M은 모나드겠죠?

In [ ]:
filter :: (a -> Bool) -> [a] -> [a]
filterM :: (Monad m) => (a -> m Bool) -> [a] -> m [a]  

-- 모나드 버전은 함수 반환형에 m 이 붙음

In [58]:
filter (< 4) [9,1,5,2,10,3]

[1,2,3]

In [69]:
keepSmall :: Int -> Writer [String] Bool  
keepSmall x  
    | x < 4 = do  
        tell ["Keeping " ++ show x]  
        return True  
    | otherwise = do  
        tell [show x ++ " is too large, throwing it away"]  
        return False  
        
fst $ runWriter $ filterM keepSmall [9,1,5,2,10,3]  
mapM_ putStrLn $ snd $ runWriter $ filterM keepSmall [9,1,5,2,10,3]  

[1,2,3]

9 is too large, throwing it away
Keeping 1
5 is too large, throwing it away
Keeping 2
10 is too large, throwing it away
Keeping 3

In [70]:
powerset :: [a] -> [[a]]
powerset xs = filterM (\x -> [True, False]) xs

Line 2: Eta reduce
Found:
powerset xs = filterM (\ x -> [True, False]) xs
Why not:
powerset = filterM (\ x -> [True, False])Line 2: Use const
Found:
\ x -> [True, False]
Why not:
const [True, False]

In [71]:
powerset [1,2,3]

[[1,2,3],[1,2],[1,3],[1],[2,3],[2],[3],[]]

### foldM

foldl에 대응하는 모나드 함수

In [ ]:
foldl :: (a -> b -> a) -> a -> [b] -> a
foldM :: (Monad m) => (a -> b -> m a) -> a -> [b] -> m a

In [72]:
foldl (\acc x -> acc + x) 0 [2,8,3,1]  

Line 1: Avoid lambda
Found:
\ acc x -> acc + x
Why not:
(+)

14

In [73]:
binSmalls :: Int -> Int -> Maybe Int
binSmalls acc x
    | x > 9 = Nothing
    | otherwise = Just (acc + x)

In [74]:
foldM binSmalls 0 [2,8,3,1]
foldM binSmalls 0 [2,11,3,1]

Just 14

Nothing

## 안전한 RPN 계산기 만들기

In [12]:
import Data.List

solveRPN :: String -> Double
solveRPN = head . foldl foldingFunction [] . words

In [2]:
readMaybe :: (Read a) => String -> Maybe a
readMaybe st = case reads st of [(x, "")] -> Just x
                                _ -> Nothing

In [4]:
readMaybe "1" :: Maybe Int
readMaybe "GOTO HELL" :: Maybe Int

Just 1

Nothing

In [16]:
foldingFunction :: [Double] -> String -> Maybe [Double]  
foldingFunction (x:y:ys) "*" = return ((x * y):ys)  
foldingFunction (x:y:ys) "+" = return ((x + y):ys)  
foldingFunction (x:y:ys) "-" = return ((y - x):ys)  
foldingFunction xs numberString = liftM (:xs) (readMaybe numberString)  

Line 5: Use fmap
Found:
liftM
Why not:
fmap

In [18]:
foldingFunction [3,2] "*"
foldingFunction [3,2] "-"
foldingFunction [] "*"
foldingFunction [] "1"
foldingFunction [] "1 wawawawa"

Just [6.0]

Just [-1.0]

Nothing

Just [1.0]

Nothing

In [19]:
import Data.List

solveRPN :: String -> Maybe Double
solveRPN st = do
    [result] <- foldM foldingFunction [] (words st)
    return result

In [20]:
solveRPN "1 2 * 4 +"
solveRPN "1 2 * 4 + 5 *"
solveRPN "1 2 * 4" -- 최종 결과물이 두개의 숫자를 가진 리스트라서 do 표현식에서 fail
solveRPN "1 8 wharglsieojf" -- readMaybe가 Nothing을 반환하므로 fail

Just 6.0

Just 30.0

Nothing

Nothing

## 모나드 함수 결합하기

In [22]:
-- <=< 는 모나드의 .
f = (+ 1) . (* 100)
f 4
g = (\x -> return (x+1)) <=< (\x -> return (x * 100))
Just 4 >>= g

401

Just 401

In [23]:
f = foldr (.) id [(+1), (*100), (+1)]
f 1

201

In [24]:
in3 start = return start >>= moveKnight >>= moveKnight >>= moveKnight
canReachIn3 :: KnightPos -> KnightPos -> Bool  
canReachIn3 start end = end `elem` in3 start  

In [ ]:
-- 모나드 결합 함수 이용하기
import Data.List

inMany :: Int -> KnightPos -> [KnightPos]
inMany x start = return start >>= foldr (<=<) return (replicate x moveKnight)
-- replicate와 <=<를 통해 원하는 횟수만큼 함수를 함성할 수 있다.

In [ ]:
canReachIn :: Int -> KnightPos -> KnightPos -> Bool
canReachIn x start end = end `elem` inMany x start

## 모나드 만들기

In [25]:
[(3,0.5),(5,0.25),(9,0.25)]
-- 숫자별 발생확률을 표시한다.

[(3,0.5),(5,0.25),(9,0.25)]

In [31]:
-- 유리수에 대한 데이터 표현
import Data.Ratio
-- 분자%분모
1%4
1%2 + 1%2
1%3 + 5%4

1 % 4

1 % 1

19 % 12

In [32]:
[(3,1%2),(5,1%4),(9,1%4)]  

[(3,1 % 2),(5,1 % 4),(9,1 % 4)]

In [34]:
-- 컨텍스트로 추가한 값을 가진다
-- 인스턴스를 만들기 위해 newtype으로 매핑한다
import Data.Ratio

newtype Prob a = Prob {getProb :: [(a,Rational)]} deriving Show
instance Functor Prob where
    fmap f (Prob xs) = Prob $ map (\(x, p) -> (f x, p)) xs

Line 7: Use first
Found:
\ (x, p) -> (f x, p)
Why not:
Data.Bifunctor.first f

In [35]:
fmap negate (Prob [(3,1%2),(5,1%4),(9,1%4)])  

Prob {getProb = [(-3,1 % 2),(-5,1 % 4),(-9,1 % 4)]}

In [36]:
flatten :: Prob (Prob a) -> Prob a
flatten (Prob xs) = Prob $ concat $ map multAll xs
    where multAll (Prob innerxs, p) = map (\(x,r) -> (x, p*r)) innerxs

Line 2: Use concatMap
Found:
concat $ map multAll xs
Why not:
concatMap multAll xsLine 3: Use second
Found:
\ (x, r) -> (x, p * r)
Why not:
Data.Bifunctor.second ((*) p)

In [44]:
import Control.Monad
import Data.Ratio

newtype Prob a = Prob {getProb :: [(a,Rational)]} deriving Show
instance Functor Prob where
    fmap f (Prob xs) = Prob $ map (\(x, p) -> (f x, p)) xs

instance Monad Prob where  
    return x = Prob [(x,1%1)]  
    m >>= f = flatten (fmap f m)  
    fail _ = Prob []  

Line 6: Use first
Found:
\ (x, p) -> (f x, p)
Why not:
Data.Bifunctor.first f

: 

In [45]:
data Coin = Heads | Tails deriving (Show, Eq)  

coin :: Prob Coin  
coin = Prob [(Heads,1%2),(Tails,1%2)]  

loadedCoin :: Prob Coin  
loadedCoin = Prob [(Heads,1%10),(Tails,9%10)]  

In [ ]:
import Data.List (all)  
  
flipThree :: Prob Bool  
flipThree = do  
    a <- coin  
    b <- coin  
    c <- loadedCoin  
    -- 비결정형 데이터의 결과인 모든 가지수가 나옴
    return (all (==Tails) [a,b,c])  
    -- 그중에서 골라낸다

In [ ]:
getProb flipThree  

-- [(False,1 % 40),(False,9 % 40),(False,1 % 40),(False,9 % 40),  
--  (False,1 % 40),(False,9 % 40),(False,1 % 40),(True,9 % 40)]  